# Actividad Guiada 3
## Autor
Miguel Ángel Álvarez Cabanes
## Github
https://github.com/maalvarezcabanes/algoritmos_optimizacion

## Importación paquetes y funciones auxiliares

In [61]:
import tsplib95
import os
import math
import random

In [63]:
file = "swiss42.tsp"
ffile = os.path.join(".", file)

problem = tsplib95.load(ffile)

In [80]:
def crear_solucion(nodos):
    ''' Creación de una solución aleatoria '''
    return [nodos[0]] +  random.sample(nodos[1:], len(nodos) - 1)

def distancia(a,b, problem):
    ''' Distancia entre dos nodos ''' 
    return problem.get_weight(a,b)

def distancia_total(solucion, problem):
    ''' Distancia total de una solución '''
    nodo_previo = solucion[0]
    for i in solucion:
        if i == nodo_previo:
            distancia_total = 0
        else:
            distancia_total += distancia(nodo_previo, i, problem)
            nodo_previo = i
    return distancia_total + distancia(solucion[-1], solucion[0], problem)

## Búsqueda aleatoria

In [81]:
def busqueda_aleatoria(problem, N):
    ''' N es el numero de iteraciones '''
    nodos = list(problem.get_nodes())

    mejor_solucion = []
    mejor_distancia = float('inf')                  #Inicializamos con un valor alto

    for i in range(N):                                #Criterio de parada: repetir N veces pero podemos incluir otros
        solucion = crear_solucion(nodos)                #Genera una solucion aleatoria
        distancia = distancia_total(solucion, problem)  #Calcula el valor objetivo(distancia total)

        if distancia < mejor_distancia:                 #Compara con la mejor obtenida hasta ahora
            mejor_solucion = solucion
            mejor_distancia = distancia

    return (mejor_solucion, mejor_distancia)

In [82]:
solucion_aleatoria, distancia_aleatoria = busqueda_aleatoria(problem, 10000)
print(f"La mejor solución es: {solucion_aleatoria} con mejor distancia {distancia_aleatoria}")

La mejor solución es: [0, 39, 21, 24, 27, 32, 29, 40, 22, 38, 25, 15, 37, 31, 5, 12, 11, 8, 18, 3, 2, 1, 4, 6, 41, 16, 10, 23, 30, 35, 20, 14, 26, 19, 7, 33, 9, 28, 17, 34, 36, 13] con mejor distancia 3691


## Búsqueda local
Para la busqueda local voy a:
1. Utilizar un multiarranque limitado a diez iteraciones (el caso general sería una limitación por tiempo).
2. Utilizar dos operadores de búsqueda local: el swap y la inserción.

In [83]:
def genera_vecina(solucion):
    ''' Generador de soluciones vecinas con: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
    Se puede modificar para aplicar otros generadores distintos que 2-opt'''

    mejor_solucion = solucion
    mejor_distancia = distancia_total(mejor_solucion, problem)
    for i in range(1,len(solucion)-1):          #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
        for j in range(i+1, len(solucion)):
            vecina = solucion.copy()
            vecina[i], vecina[j] = vecina[j], vecina[i]

            #Se evalua la nueva solución ...
            distancia_vecina = distancia_total(vecina, problem)

            #... para guardarla si mejora las anteriores
            if distancia_vecina <= mejor_distancia:
                mejor_distancia = distancia_vecina
                mejor_solucion = vecina

    return (mejor_solucion, mejor_distancia)

print(f"Distancia Solucion Inicial: {distancia_aleatoria}")

solucion_vecina, distancia_vecina = genera_vecina(solucion_aleatoria)
print(f"Distancia Mejor Solucion Local: {distancia_vecina}")

Distancia Solucion Inicial: 3691
Distancia Mejor Solucion Local: 3336


In [95]:
def genera_insercion(solucion):
    mejor_solucion = solucion
    mejor_distancia = distancia_total(mejor_solucion, problem)
    for i in range(1,len(solucion)-1):          #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
        for j in range(i+1, len(solucion)):
            vecina = solucion.copy()
            aux = vecina.pop(i)
            vecina.insert(j, aux)

            #Se evalua la nueva solución ...
            distancia_vecina = distancia_total(vecina, problem)

            #... para guardarla si mejora las anteriores
            if distancia_vecina <= mejor_distancia:
                mejor_distancia = distancia_vecina
                mejor_solucion = vecina

    return (mejor_solucion, mejor_distancia)

print(f"Distancia Solucion Inicial: {distancia_aleatoria}")

solucion_insercion, distancia_insercion = genera_insercion(solucion_aleatoria)
print(f"Distancia Mejor Solucion Local: {distancia_insercion}")

Distancia Solucion Inicial: 3691
Distancia Mejor Solucion Local: 3412


In [96]:
def busqueda_local(problem, iteraciones):
    mejor_solucion = []
    mejor_solucion_total = []
    mejor_distancia_total = None
    
    ops = [genera_vecina, genera_insercion]

    for i in range(iteraciones):
        #Generar una solucion inicial de referencia(aleatoria)
        solucion_referencia = crear_solucion(nodos)
        mejor_distancia = distancia_total(solucion_referencia, problem)

        for op in ops:
            while(1):
                #Obtenemos la mejor vecina ...
                vecina, distancia_vecina = op(solucion_referencia)

                #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
                if distancia_vecina < mejor_distancia:
                    mejor_solucion, mejor_distancia = vecina, distancia_vecina  #Guarda la mejor solución encontrada
                    solucion_referencia = vecina
                else:
                    if (mejor_distancia_total == None) or (mejor_distancia < mejor_distancia_total):
                        mejor_solucion_total, mejor_distancia_total = (mejor_solucion, mejor_distancia)
                    break
                
    return (mejor_solucion_total, mejor_distancia_total)

In [97]:
solucion_local, distancia_local = busqueda_local(problem, 3)
print(f"La mejor solución encontrada es: {solucion_local} con distancia {distancia_local}")

La mejor solución encontrada es: [0, 34, 33, 20, 35, 36, 31, 17, 37, 15, 16, 14, 7, 32, 38, 22, 24, 40, 21, 39, 8, 10, 12, 11, 25, 41, 23, 9, 29, 30, 28, 2, 27, 3, 4, 18, 26, 13, 19, 5, 6, 1] con distancia 1475
